In [10]:
import numpy as np
import pandas as pd
import random
import math
from scipy.stats import norm

In [6]:
# question 1.1
# parameters
a0 = 2.50662823884    
b0 = -8.47351093090
a1 = -18.61500062529
b1 = 23.08336743743
a2 = 41.39119773534
b2 = -21.06224101826
a3 = -25.44106049637
b3 = 3.13082909833
c0 = 0.3374754822726147
c5 = 0.0003951896511919
c1 = 0.9761690190917186
c6 = 0.0000321767881768
c2 = 0.1607979714918209
c7 = 0.0000002888167364
c3 = 0.0276438810333863
c8 = 0.0000003960315187
c4 = 0.0038405729373609

# Beasley–Springer–Moro algorithm function
def bsm(u: float):
    """
    intput: u is between 0 and 1
    output: x, approximation to \phi^-1(u)
    """
    y = u - 0.5
    if abs(y) < 0.42:
        r = y * y
        x = y * (((a3 * r + a2) * r + a1) * r + a0) / ((((b3 * r + b2) * r + b1) * r + b0) * r + 1)
    else:
        r = u
        if y > 0: r = 1 - u
        r = math.log(-math.log(r))
        x = c0 + r * (c1 + r * (c2 + r * (c3 + r * (c4+ r * (c5 + r * (c6 + r * (c7 + r * c8)))))))
        if y < 0: 
            x = -x
    return x
        

In [12]:
# question 1.2
def black_scholes(S, K, T, r, q, sigma, option_type):

    d1 = (np.log(S / K) + (r - q + (sigma ** 2) / 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if option_type == 'call':
        option_price = S * np.exp(-q * T) * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
        delta = np.exp(-q * T) * norm.cdf(d1)
    elif option_type == 'put':
        option_price = K * np.exp(-r * T) * norm.cdf(-d2) - S * np.exp(-q * T) * norm.cdf(-d1)
        delta = -np.exp(-q * T) * norm.cdf(-d1)
    else:
        raise ValueError("Invalid option type. Use 'call' or 'put'.")

    gamma = np.exp(-q * T) * norm.pdf(d1) / (S * sigma * np.sqrt(T))
    vega = S * np.exp(-q * T) * norm.pdf(d1) * np.sqrt(T)

    return option_price, delta, gamma, vega

In [ ]:
"""
Consider an asset with spot price $50 and spot volatility 30% (i.e., with intital variance V0 = 0.09),
and assume that the price of the asset follows a Heston Model with three month reversion time scale to
a long–term standard deviation mean of 35% and with 25% standard deviation of the variance, i.e.,
λ = 4; sqrt(V) = 0.25, η = 0.25.
Assume a correlation rho = -0.15 between the Wiener processes.
Consider a six months at–the–money put on this option. Let T = 0.5 and K = 50. Assume the
risk-free rate is constant r = 0.05.
Simulate n random paths for the evolution of the price of the asset as above, each one discretized by m
time steps corresponding to δt = T /m. Generate N = 2nm independent samples of the standard normal
distribution using the linear congruential generator and the Box–Muller method as before.
Use a small time interval when discretizing the path, i.e., half a day, in order to capture the mean
reversion of the volatility. This corresponds approximately to m = 175. Use n = 500 · 2^k paths, where
k = 0 : 5, i.e., N = 2 · 87500 · 2^5
Let Vk be the value of the option corresponding to the asset evolving along the path k, for k = 1 : n,
and let \hat{V}(n) = \sum_{i=1}^n V_i/n
For each value of Vb(n) also compute the corresponding Black–Scholes implied volatility

"""

import numpy as np
import pandas as pd


# question 1.4
def calculate_option_price(S0, K, T, r, q, sigma, delta_s, num_simulations):
    z = np.random.normal(0, 1, num_simulations)
    # stock price
    ST = S0 * np.exp((r - q - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * z)
    ST_plus = (S0 + delta_s) * np.exp((r - q - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * z)
    ST_minus = (S0 - delta_s) * np.exp((r - q - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * z)
    # option price
    P = np.exp(-r*T) * (K-ST)
    P[P < 0] = 0
    return P.mean()


    option_price = np.mean(option_payoff)
    option_payoff_plus = np.max(ST_plus - K, 0)
    option_price_plus = np.exp(-r * T) * np.mean(option_payoff_plus)
    option_payoff_minus = np.max(ST_m

In [35]:
# question 1.4
def calculate_option_price(S0, K, T, r, q, sigma, delta_s, num_simulations):
    np.random.seed(42)
    z = np.random.normal(0, 1, num_simulations)
    # stock price
    ST = S0 * np.exp((r - q - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * z)
    ST_plus = (S0 + delta_s) * np.exp((r - q - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * z)
    ST_minus = (S0 - delta_s) * np.exp((r - q - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * z)
    # option price
    option_payoff = np.exp(-r * T) * (ST - K)
    option_payoff[option_payoff < 0] = 0
    option_price = np.mean(option_payoff)
    option_payoff_plus = np.exp(-r * T) * (ST_plus - K)
    option_payoff_plus[option_payoff_plus < 0] = 0
    option_price_plus = np.mean(option_payoff_plus)
    option_payoff_minus = np.exp(-r * T) * (ST_minus - K)
    option_payoff_minus[option_payoff_minus < 0] = 0
    option_price_minus = np.mean(option_payoff_minus)
    return option_price, option_price_plus, option_price_minus

# parameters
S0 = 100  # Current stock price
K = 100  # Strike price
T = 1    # Time to expiration (in years)
r = 0.05 # Risk-free interest rate
sigma = 0.2 # Volatility
q = 0.02  # Dividend yield

# bs value and delta
bsm_value, bsm_delta, bsm_gamma, bsm_vega = black_scholes(S0, K, T, r, q, sigma, "call")
print("BSM value:")
print(bsm_value, bsm_delta, bsm_gamma)

delta_s = 0.1 # delta_s

result_df = pd.DataFrame(columns=['Delta', 'Delta_diff', 'Gamma', 'Gamma_diff'])
for k in range(10):
    N = 10000 * (2**k)
    # option price
    option_price, option_price_plus, option_price_minus = calculate_option_price(S0, K, T, r, q, sigma, delta_s, N)
    # delta calculation
    delta = (option_price_plus - option_price_minus) / (2 * delta_s)
    # gamma calculation
    gamma = (option_price_plus - 2 * option_price + option_price_minus) / (delta_s)**2
    # diff from bsm value
    delta_diff = N**0.5 * abs(delta - bsm_delta)
    gamma_diff = N**0.5 * abs(gamma - bsm_gamma)
    # add to result_df
    result_df = result_df.append({'Delta': delta, 'Delta_diff': delta_diff, 'Gamma': gamma, 'Gamma_diff': gamma_diff}, ignore_index=True)

print("Monte Carlo value:")
print(result_df)

BSM value:
9.227005508154036 0.586851146134764 0.018950578755008718
Monte Carlo value:
      Delta  Delta_diff     Gamma  Gamma_diff
0  0.586687    0.000164  0.020190    0.123937
1  0.590236    0.003385  0.018756    0.027557
2  0.586199    0.000652  0.020305    0.270880
3  0.588093    0.001242  0.020660    0.483445
4  0.587782    0.000931  0.020448    0.599046
5  0.586239    0.000612  0.019418    0.264689
6  0.586397    0.000454  0.019043    0.074077
7  0.586354    0.000497  0.019098    0.167173
8  0.586890    0.000039  0.019086    0.216262
9  0.586879    0.000028  0.019060    0.247508
